# Pokemon Data Analysis

Using [PokéAPI](https://pokeapi.co/), I will create a data pipeline in python to extract and transform data about Pokémon in a MySQL database. From there, I’ll analysis different aspects of the data in python and SQL

In [1]:
import pokemon_pipline as pk
import pandas as pd

### 1. Collect and Format Data

Firstly, by using the requests package and a Pokémon object, I will create a list containing the information of every Pokémon, then convert the list into a pandas DataFrame as a more easily accessible form.

In [2]:
pokemon_base_url: str = "https://pokeapi.co/api/v2/pokemon/"
pokemon_list: list = pk.import_pokemon_data_to_list(4,url = pokemon_base_url)

pokemon_df: pd.DataFrame = pk.list_pokemon_data_to_pd(pokemon_list)
pokemon_df.head()

,pokemonId,name,type1,type2,height,weight,hp,attack,defense,special_attack,special_defense,speed
0,1,bulbasaur,grass,poison,7,69,45,49,49,65,65,45
1,2,ivysaur,grass,poison,10,130,60,62,63,80,80,60
2,3,venusaur,grass,poison,20,1000,80,82,83,100,100,80
3,4,charmander,fire,None,6,85,39,52,43,60,50,65


I will do the same for Pokémon types, ending up with a DataFrame of Pokémon types and a corresponding ID


In [7]:
type_base_url: str = "https://pokeapi.co/api/v2/type/"
pokemon_types: list = pk.import_type_data_to_list(url = type_base_url)

types_df: pd.DataFrame = pk.list_types_data_to_pd(pokemon_types)
types_df.head()

,pokemonType,typeID
0,normal,0
1,fighting,1
2,flying,2
3,poison,3
4,ground,4


### 2. Export to Data Storage and Data Base

Can export the pokemon data now in the form of a csv and parquet file

In [ ]:
pk.data_pd_to_csv(pokemon_df)
pk.data_pd_to_parquet(pokemon_df)


![Pokémon Database erd](PokemonDB_erd.png)

Data in pandas and export data into sql db and pasquel

get data from sql in each query or just use pandas

for each question:  
type   
height/weight    
stats